## Getting a deeper understanding of what tokenizer does inside training loop like ORPO 

In [1]:
import torch
from transformers import pipeline, Conversation, AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from peft import PeftModel

torch.random.manual_seed(0)

# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model_id = "/home/stefanwebb/models/llm/meta_llama3-8b"
tokenizer = AutoTokenizer.from_pretrained(model_id)

2024-06-21 14:27:17.811372: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 14:27:18.117047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 14:27:18.870479: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# These were captured from ORPOTrainer.compute_loss
chosen_input_ids = [128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555,
            922,  26110,    848,   1976,     88,    323,   6070,   8331,     30,
         128009, 128006,  78191, 128007,    271,  82038,    848,   1976,     88,
            374,    279,   8198,    315,    279,  27204,    315,  33299,    304,
          82486,     13,   1102,    374,    264,  13057,    323,  88419,   2115,
            430,    706,   8522,    304,  22027,     11,  30903,     11,   7384,
           8198,     11,  34458,     11,    323,  15009,    382,    791,   6070,
           8331,    374,    264,  37072,    734,    430,    374,   1511,    311,
           7664,    279,   3722,  16597,    315,  17301,    555,    264,  26110,
             13,   1102,    374,    264,   6485,   1396,    430,    374,   5552,
            311,    279,  25524,  10093,    304,    279,  26110,    382,    791,
           6070,   8331,    649,    387,   1511,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
            382,  82038,    848,   1976,     88,    374,    264,   8147,   5507,
            369,   8830,    279,   6070,    315,   7384,     13,   1102,    706,
           1027,   1511,    311,   8417,    279,  14726,    315,   1690,   3062,
           7384,     11,   2737,  37182,     11,   5347,   1965,   1076,   1105,
             11,    323,  35410,     82,     13,   1102,    374,   1101,   1511,
            311,   4007,    279,   6070,    315,  24156,   7384,     11,   1778,
            439,  28896,    323,  15922,    382,    791,   6070,   8331,    374,
            264,   1401,   7434,    304,  26110,    848,   1976,     88,     13,
           1102,    374,   1511,    311,   7664,    279,   3722,  16597,    315,
          17301,    555,    264,  26110,    323,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
             13, 128009, 128001]

chosen_labels = [  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100, 128006,  78191, 128007,    271,  82038,    848,   1976,     88,
            374,    279,   8198,    315,    279,  27204,    315,  33299,    304,
          82486,     13,   1102,    374,    264,  13057,    323,  88419,   2115,
            430,    706,   8522,    304,  22027,     11,  30903,     11,   7384,
           8198,     11,  34458,     11,    323,  15009,    382,    791,   6070,
           8331,    374,    264,  37072,    734,    430,    374,   1511,    311,
           7664,    279,   3722,  16597,    315,  17301,    555,    264,  26110,
             13,   1102,    374,    264,   6485,   1396,    430,    374,   5552,
            311,    279,  25524,  10093,    304,    279,  26110,    382,    791,
           6070,   8331,    649,    387,   1511,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
            382,  82038,    848,   1976,     88,    374,    264,   8147,   5507,
            369,   8830,    279,   6070,    315,   7384,     13,   1102,    706,
           1027,   1511,    311,   8417,    279,  14726,    315,   1690,   3062,
           7384,     11,   2737,  37182,     11,   5347,   1965,   1076,   1105,
             11,    323,  35410,     82,     13,   1102,    374,   1101,   1511,
            311,   4007,    279,   6070,    315,  24156,   7384,     11,   1778,
            439,  28896,    323,  15922,    382,    791,   6070,   8331,    374,
            264,   1401,   7434,    304,  26110,    848,   1976,     88,     13,
           1102,    374,   1511,    311,   7664,    279,   3722,  16597,    315,
          17301,    555,    264,  26110,    323,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
             13, 128009, 128001]

rejected_input_ids = [128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555,
            922,  26110,    848,   1976,     88,    323,   6070,   8331,     30,
         128009, 128006,  78191, 128007,    271,  96556,      0,  29016,    848,
           1976,     88,    374,    279,   4007,    315,    279,   6070,     11,
          27204,    315,  33299,     11,    323,   6012,    315,  48473,     13,
          29696,   8331,     11,    389,    279,   1023,   1450,     11,    374,
            264,  37072,   5852,    430,  16964,    279,  27204,    315,  33299,
            477,  35715,    304,    264,  26110,     13,   1102,    374,   1511,
            311,   8417,    279,  26110,    848,  79173,   6012,    315,    264,
          26110,     11,   1778,    439,   1202,   5089,   2849,  15696,    323,
          46220,    382,   2746,    499,    617,    904,   3230,   4860,    922,
          26110,    848,   1976,     88,    477,   6070,   9547,     11,    358,
           4265,    387,   6380,    311,   1520,      0, 128009, 128001]

rejected_labels = [  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100, 128006,  78191, 128007,    271,  96556,      0,  29016,    848,
           1976,     88,    374,    279,   4007,    315,    279,   6070,     11,
          27204,    315,  33299,     11,    323,   6012,    315,  48473,     13,
          29696,   8331,     11,    389,    279,   1023,   1450,     11,    374,
            264,  37072,   5852,    430,  16964,    279,  27204,    315,  33299,
            477,  35715,    304,    264,  26110,     13,   1102,    374,   1511,
            311,   8417,    279,  26110,    848,  79173,   6012,    315,    264,
          26110,     11,   1778,    439,   1202,   5089,   2849,  15696,    323,
          46220,    382,   2746,    499,    617,    904,   3230,   4860,    922,
          26110,    848,   1976,     88,    477,   6070,   9547,     11,    358,
           4265,    387,   6380,    311,   1520,      0, 128009, 128001]

prompt_input_ids = [128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555, 922,  26110,    848,   1976,     88,    323,   6070,   8331,    30, 128009]

In [10]:
"""
chosen_input_ids
chosen_labels
rejected_input_ids
rejected_labels
prompt_input_ids
"""
prompt_input_str = tokenizer.decode(prompt_input_ids)
chosen_input_str = tokenizer.decode(chosen_input_ids)
rejected_input_str = tokenizer.decode(rejected_input_ids)


In [12]:
# NOTE: Chosen input and rejected input begin with prompt followed by response!
print('prompt_input_ids\n', prompt_input_str)
print('\nchosen_input_ids\n', chosen_input_str)
print('\nrejected_input_ids\n', rejected_input_str)

prompt_input_ids
 <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Do you know something about crystallography and structure factor?<|eot_id|>

chosen_input_ids
 <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Do you know something about crystallography and structure factor?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Crystallography is the science of the arrangement of atoms in solids. It is a vast and interdisciplinary field that has applications in physics, chemistry, materials science, biology, and engineering.

The structure factor is a mathematical function that is used to describe the diffraction of waves by a crystal. It is a complex number that is related to the atomic positions in the crystal.

The structure factor can be used to calculate the intensity of the diffracted waves. This information can be used to determine the atomic positions in the crystal and to study the structure of materials.

Crystallography is a powerful tool for understan

In [15]:
chosen_labels_str = tokenizer.decode([x if x != -100 else 128002 for x in chosen_labels])
rejected_labels_str = tokenizer.decode([x if x != -100 else 128002 for x in rejected_labels])

In [16]:
print('chosen_labels_str\n', chosen_labels_str)
print('\nrejected_labels_str\n', rejected_labels_str)

chosen_labels_str
 <|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|start_header_id|>assistant<|end_header_id|>

Crystallography is the science of the arrangement of atoms in solids. It is a vast and interdisciplinary field that has applications in physics, chemistry, materials science, biology, and engineering.

The structure factor is a mathematical function that is used to describe the diffraction of waves by a crystal. It is a complex number that is related to the atomic positions in the crystal.

The stru

In [17]:
concatenated_input_ids = torch.tensor([[128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555,
            922,  26110,    848,   1976,     88,    323,   6070,   8331,     30,
         128009, 128006,  78191, 128007,    271,  82038,    848,   1976,     88,
            374,    279,   8198,    315,    279,  27204,    315,  33299,    304,
          82486,     13,   1102,    374,    264,  13057,    323,  88419,   2115,
            430,    706,   8522,    304,  22027,     11,  30903,     11,   7384,
           8198,     11,  34458,     11,    323,  15009,    382,    791,   6070,
           8331,    374,    264,  37072,    734,    430,    374,   1511,    311,
           7664,    279,   3722,  16597,    315,  17301,    555,    264,  26110,
             13,   1102,    374,    264,   6485,   1396,    430,    374,   5552,
            311,    279,  25524,  10093,    304,    279,  26110,    382,    791,
           6070,   8331,    649,    387,   1511,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
            382,  82038,    848,   1976,     88,    374,    264,   8147,   5507,
            369,   8830,    279,   6070,    315,   7384,     13,   1102,    706,
           1027,   1511,    311,   8417,    279,  14726,    315,   1690,   3062,
           7384,     11,   2737,  37182,     11,   5347,   1965,   1076,   1105,
             11,    323,  35410,     82,     13,   1102,    374,   1101,   1511,
            311,   4007,    279,   6070,    315,  24156,   7384,     11,   1778,
            439,  28896,    323,  15922,    382,    791,   6070,   8331,    374,
            264,   1401,   7434,    304,  26110,    848,   1976,     88,     13,
           1102,    374,   1511,    311,   7664,    279,   3722,  16597,    315,
          17301,    555,    264,  26110,    323,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
             13, 128009, 128001],
        [128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555,
            922,  26110,    848,   1976,     88,    323,   6070,   8331,     30,
         128009, 128006,  78191, 128007,    271,  96556,      0,  29016,    848,
           1976,     88,    374,    279,   4007,    315,    279,   6070,     11,
          27204,    315,  33299,     11,    323,   6012,    315,  48473,     13,
          29696,   8331,     11,    389,    279,   1023,   1450,     11,    374,
            264,  37072,   5852,    430,  16964,    279,  27204,    315,  33299,
            477,  35715,    304,    264,  26110,     13,   1102,    374,   1511,
            311,   8417,    279,  26110,    848,  79173,   6012,    315,    264,
          26110,     11,   1778,    439,   1202,   5089,   2849,  15696,    323,
          46220,    382,   2746,    499,    617,    904,   3230,   4860,    922,
          26110,    848,   1976,     88,    477,   6070,   9547,     11,    358,
           4265,    387,   6380,    311,   1520,      0, 128009, 128001, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002, 128002,
         128002, 128002, 128002]], device='cuda:0')

In [18]:
concatenated_attention_mask = torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [19]:
attention_mask = torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [20]:
labels = torch.tensor([[128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555,
            922,  26110,    848,   1976,     88,    323,   6070,   8331,     30,
         128009, 128006,  78191, 128007,    271,  82038,    848,   1976,     88,
            374,    279,   8198,    315,    279,  27204,    315,  33299,    304,
          82486,     13,   1102,    374,    264,  13057,    323,  88419,   2115,
            430,    706,   8522,    304,  22027,     11,  30903,     11,   7384,
           8198,     11,  34458,     11,    323,  15009,    382,    791,   6070,
           8331,    374,    264,  37072,    734,    430,    374,   1511,    311,
           7664,    279,   3722,  16597,    315,  17301,    555,    264,  26110,
             13,   1102,    374,    264,   6485,   1396,    430,    374,   5552,
            311,    279,  25524,  10093,    304,    279,  26110,    382,    791,
           6070,   8331,    649,    387,   1511,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
            382,  82038,    848,   1976,     88,    374,    264,   8147,   5507,
            369,   8830,    279,   6070,    315,   7384,     13,   1102,    706,
           1027,   1511,    311,   8417,    279,  14726,    315,   1690,   3062,
           7384,     11,   2737,  37182,     11,   5347,   1965,   1076,   1105,
             11,    323,  35410,     82,     13,   1102,    374,   1101,   1511,
            311,   4007,    279,   6070,    315,  24156,   7384,     11,   1778,
            439,  28896,    323,  15922,    382,    791,   6070,   8331,    374,
            264,   1401,   7434,    304,  26110,    848,   1976,     88,     13,
           1102,    374,   1511,    311,   7664,    279,   3722,  16597,    315,
          17301,    555,    264,  26110,    323,    311,  11294,    279,  21261,
            315,    279,   1422,   1658,  22955,  17301,     13,   1115,   2038,
            649,    387,   1511,    311,   8417,    279,  25524,  10093,    304,
            279,  26110,    323,    311,   4007,    279,   6070,    315,   7384,
             13, 128009, 128001],
        [128000, 128006,    882, 128007,    271,   5519,    499,   1440,   2555,
            922,  26110,    848,   1976,     88,    323,   6070,   8331,     30,
         128009, 128006,  78191, 128007,    271,  96556,      0,  29016,    848,
           1976,     88,    374,    279,   4007,    315,    279,   6070,     11,
          27204,    315,  33299,     11,    323,   6012,    315,  48473,     13,
          29696,   8331,     11,    389,    279,   1023,   1450,     11,    374,
            264,  37072,   5852,    430,  16964,    279,  27204,    315,  33299,
            477,  35715,    304,    264,  26110,     13,   1102,    374,   1511,
            311,   8417,    279,  26110,    848,  79173,   6012,    315,    264,
          26110,     11,   1778,    439,   1202,   5089,   2849,  15696,    323,
          46220,    382,   2746,    499,    617,    904,   3230,   4860,    922,
          26110,    848,   1976,     88,    477,   6070,   9547,     11,    358,
           4265,    387,   6380,    311,   1520,      0, 128009, 128001,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100]], device='cuda:0')

In [23]:
concatenated_input_strs = [tokenizer.decode(x) for x in concatenated_input_ids]

In [24]:
concatenated_input_strs

['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nDo you know something about crystallography and structure factor?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nCrystallography is the science of the arrangement of atoms in solids. It is a vast and interdisciplinary field that has applications in physics, chemistry, materials science, biology, and engineering.\n\nThe structure factor is a mathematical function that is used to describe the diffraction of waves by a crystal. It is a complex number that is related to the atomic positions in the crystal.\n\nThe structure factor can be used to calculate the intensity of the diffracted waves. This information can be used to determine the atomic positions in the crystal and to study the structure of materials.\n\nCrystallography is a powerful tool for understanding the structure of materials. It has been used to determine the structures of many important materials, including metals, semiconductors, and pharmaceuticals